# **TALLER 2 - Modelo Random Forest**

* Daniel Felipe Vargas Ulloa
* Andrés Francisco Borda Rincón

## **1. Preparación**

Para la realización de este taller, se importan automaticamente las librerías necesarias y se descargará el conjunto de datos directamente del repositorio de Grocery Store Dataset donde se encuentra alojado. Para este ultimo paso es vital tener conexión a internet y tener git instalado en el entorno de trabajo como se especificó en la descripción del repositorio.

**NOTA:** La ejecución de este notebook require al menos 14GB de RAM **disponible**. De lo contrario se encontrará con un error de out of memory error.

### **1.1 Importar librerías y descargar datos**

In [22]:
# Install all required libraries if not already installed using pip
import subprocess
import sys

!{sys.executable} -m pip install -r requirements.txt | grep -v 'already satisfied'

In [23]:
# pandas
import pandas as pd

# numpy version before 2.0.0
import numpy as np

# matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# tensorflow
import imgaug as ia

# os
import os

# Image manipulation
from PIL import Image
import imgaug.augmenters as iaa
import imageio.v2 as imageio

In [24]:
# ensures the root directory is not the python path but the root of the project 'MINE-4101-Taller-2'
if os.path.basename(os.getcwd()) == 'src':
    os.chdir('..')

In [25]:
# Clona el repositorio de github con el dataset si no existe ya en el entorno

if not os.path.exists('GroceryStoreDataset'):
    !git clone https://github.com/marcusklasson/GroceryStoreDataset.git

# Carga el dataset
df = pd.read_csv('GroceryStoreDataset/dataset/classes.csv')

In [26]:
# Configuración de pandas para extender el número de filas y columnas mostradas
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', 200)
pd.set_option("display.max_rows", 100)
pd.set_option('expand_frame_repr', False)
np.set_printoptions(threshold=sys.maxsize)

### **1.2. Importar datos aumentados**

Ademas debemos realizar brevemente las transformaciones realizadas anterioremente en el notebook de preparación de datos para poder realizar el modelo Random Forest. En caso de que ya se haya realizado este proceso, se omite automáticamente.

In [27]:
# Cargar la lista de imagenes de cada uno de los productos, por ahoar se usará granularidad de prpducto y no de tipo de producto

# Carga los 3 datasets, GroceryStoreDataset/test, GroceryStoreDataset/train y GroceryStoreDataset/val

# Define the column names
column_names = ['Image Path', 'Class ID', 'Coarse Class ID']

# Carga el dataset de entrenamiento
df_train = pd.read_csv('GroceryStoreDataset/dataset/train.txt', names=column_names, header=None)

# Carga el dataset de validación
df_val = pd.read_csv('GroceryStoreDataset/dataset/val.txt', names=column_names, header=None)

# Carga el dataset de prueba
df_test = pd.read_csv('GroceryStoreDataset/dataset/test.txt', names=column_names, header=None)

# Print the first few rows of each dataframe to verify

In [28]:
# Añade 'GroceryStoreDataset/dataset' al comienzo de cada ruta de imagen
df_train['Image Path'] = 'GroceryStoreDataset/dataset/' + df_train['Image Path']
df_val['Image Path'] = 'GroceryStoreDataset/dataset/' + df_val['Image Path']
df_test['Image Path'] = 'GroceryStoreDataset/dataset/' + df_test['Image Path']

# Check if the directory already exists
if os.path.exists('GroceryStoreDataset/dataset/train_128x128'):
    print("Directory already exists. Skipping the process.")
else:

    # Create the directory
    os.makedirs('GroceryStoreDataset/dataset/train_128x128', exist_ok=True)

    # Function to resize and save the image
    def resize_and_save(image_path, size, save_path):
        with Image.open(image_path) as img:
            img_resized = img.resize(size)
            img_resized.save(save_path)

    # Create a dictionary to map class_id and coarse_class_id to their names
    class_id_to_name = df.set_index('Class ID (int)')['Class Name (str)'].to_dict()
    coarse_class_id_to_name = df.set_index('Coarse Class ID (int)')['Coarse Class Name (str)'].to_dict()

    # Resize and save all images in the train dataset
    for index, row in df_train.iterrows():
        image_path = row['Image Path']
        class_id = row['Class ID']
        coarse_class_id = row['Coarse Class ID']
        class_name = class_id_to_name[class_id]
        coarse_class_name = coarse_class_id_to_name[coarse_class_id]
        image_name = image_path.split('/')[-1]
        save_path = f'GroceryStoreDataset/dataset/train_128x128/{coarse_class_name}/{class_name}/{image_name}'
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        resize_and_save(image_path, (128, 128), save_path)

    # Check the number of images in the resized directory, it should be the same as the original directory

    # Count the number of images in the original directory
    original_image_count = sum([len(files) for r, d, files in os.walk('GroceryStoreDataset/dataset/train')])
    print(f"Number of images in the original directory: {original_image_count}")

    # Count the number of images in the resized directory
    resized_image_count = sum([len(files) for r, d, files in os.walk('GroceryStoreDataset/dataset/train_128x128')])
    print(f"Number of images in the resized directory: {resized_image_count}")

    # Check if the number of images in the original and resized directories are the same
    if original_image_count == resized_image_count:
        print("Number of images in the original and resized directories match.")
    else:
        print("Number of images in the original and resized directories do not match.")

Directory already exists. Skipping the process.


In [29]:
# Define the augmentation techniques
augmenters = [
    iaa.Fliplr(),  # Horizontal flip
    iaa.Flipud(),  # Vertical flip
    iaa.Crop(percent=(0.2)),  # Crop
    iaa.Multiply((1.3)),  # Brightness
    iaa.GaussianBlur(sigma=(1.2)),  # Blur
    iaa.Affine(rotate=(53), mode='reflect'),
    iaa.PerspectiveTransform(scale=(0.13))
]

# Reemplaza train en los paths con train_128x128
df_train['Image Path'] = df_train['Image Path'].str.replace('train/', 'train_128x128/')

# Remove Fruit/, Vegerable/ and Packages from the image path
df_train['Image Path'] = df_train['Image Path'].str.replace('/Fruit/', '/')
df_train['Image Path'] = df_train['Image Path'].str.replace('/Vegetables/', '/')
df_train['Image Path'] = df_train['Image Path'].str.replace('/Packages/', '/')

# Check if the directory already exists
if os.path.exists('GroceryStoreDataset/dataset/train_augmented'):
    print("Directory already exists. Skipping the process.")
else:
    # Create the directory
    os.makedirs('GroceryStoreDataset/dataset/train_augmented', exist_ok=True)

    # Function to apply augmentations and save the image
    def augment_and_save(image_path, augmenters, save_dir, image_name):
      try:
        with Image.open(image_path) as img:
            img_np = np.array(img)
            # Save the original image
            original_save_path = os.path.join(save_dir, f'original_{image_name}')
            img.save(original_save_path)
            # Apply augmentations and save the augmented images
            for i, augmenter in enumerate(augmenters):
                augmented_img_np = augmenter(image=img_np)
                augmented_img = Image.fromarray(augmented_img_np)
                augmented_save_path = os.path.join(save_dir, f'augmented_{i}_{image_name}')
                augmented_img.save(augmented_save_path)

      except FileNotFoundError:
        # Modify the path by repeating the subfolder name
        base_path = 'GroceryStoreDataset/dataset/train_128x128/'
        if base_path in image_path:
            path_parts = image_path.split(base_path)
            subfolder, remaining_path = path_parts[1].split('/', 1)
            new_image_url = os.path.join(path_parts[0], base_path, subfolder, subfolder, remaining_path)

            try:
              with Image.open(new_image_url) as img:
                img_np = np.array(img)
                # Save the original image
                original_save_path = os.path.join(save_dir, f'original_{image_name}')
                img.save(original_save_path)
                # Apply augmentations and save the augmented images
                for i, augmenter in enumerate(augmenters):
                    augmented_img_np = augmenter(image=img_np)
                    augmented_img = Image.fromarray(augmented_img_np)
                    augmented_save_path = os.path.join(save_dir, f'augmented_{i}_{image_name}')
                    augmented_img.save(augmented_save_path)
            except:
              # raise FileNotFoundError(f"File not found at either original or modified path: {image_path}")
              if subfolder == 'Brown-Cap-Mushroom':
                  new_image_url = os.path.join(path_parts[0], base_path, 'Mushroom', subfolder, remaining_path)
                  with Image.open(new_image_url) as img:
                    img_np = np.array(img)
                    # Save the original image
                    original_save_path = os.path.join(save_dir, f'original_{image_name}')
                    img.save(original_save_path)
                    # Apply augmentations and save the augmented images
                    for i, augmenter in enumerate(augmenters):
                        augmented_img_np = augmenter(image=img_np)
                        augmented_img = Image.fromarray(augmented_img_np)
                        augmented_save_path = os.path.join(save_dir, f'augmented_{i}_{image_name}')
                        augmented_img.save(augmented_save_path)


        else:
          raise FileNotFoundError(f"File path does not contain expected base path: {image_path}")



    # Create a dictionary to map class_id and coarse_class_id to their names
    class_id_to_name = df.set_index('Class ID (int)')['Class Name (str)'].to_dict()
    coarse_class_id_to_name = df.set_index('Coarse Class ID (int)')['Coarse Class Name (str)'].to_dict()

    # Apply augmentations and save all images in the train dataset
    for index, row in df_train.iterrows():
        image_path = row['Image Path']
        class_id = row['Class ID']
        coarse_class_id = row['Coarse Class ID']
        class_name = class_id_to_name[class_id]
        coarse_class_name = coarse_class_id_to_name[coarse_class_id]
        image_name = image_path.split('/')[-1]
        save_dir = f'GroceryStoreDataset/dataset/train_augmented/{coarse_class_name}/{class_name}'
        os.makedirs(save_dir, exist_ok=True)
        augment_and_save(image_path, augmenters, save_dir, image_name)

    # Check the number of images in the augmented directory, it should be the same as the original directory

    # Count the number of images in the original directory
    original_image_count = sum([len(files) for r, d, files in os.walk('GroceryStoreDataset/dataset/train')])
    print(f"Number of images in the original directory: {original_image_count}")

    # Count the number of images in the augmented directory
    augmented_image_count = sum([len(files) for r, d, files in os.walk('GroceryStoreDataset/dataset/train_augmented')])
    print(f"Number of images in the augmented directory: {augmented_image_count}")

    # Check if the number of images in the original and augmented directories are the same by a factor of 8
    if original_image_count * 8 == augmented_image_count:
        print("Number of images in the original and augmented directories match by a factor of 8.")
    else:
        print("Number of images in the original and augmented directories do not match.")


Directory already exists. Skipping the process.


In [30]:
import pandas as pd


# Function to generate augmented image paths
def generate_augmented_paths(image_path):

    base_path = image_path.replace('train/', 'train_augmented/')

    # remove 'Fruit', 'Packages' and 'Vegeragles' as we're no longer grouping by type of product
    base_path = base_path.replace('/Fruit/', '/')
    base_path = base_path.replace('/Packages/', '/')
    base_path = base_path.replace('/Vegetables/', '/')

    # augmented image have the same path but have 'augmented_x_' prefix before the image name, so thats after the last '/'
    # for example: 'train_augmented/Apple/Granny-Smith/Golden-Delicious_001.jpg' -> 'train_augmented/Apple/Granny-Smith/augmented_0_Golden-Delicious_001.jpg'
    augmented_paths = []
    for i in range(7):
        augmented_path = base_path.rsplit('/', 1)[0] + f'/augmented_{i}_' + base_path.rsplit('/', 1)[1]
        augmented_paths.append(augmented_path)

    # also include the original image path
    # for example: 'train_augmented/Apple/Granny-Smith/Golden-Delicious_001.jpg' -> 'train_augmented/Apple/Granny-Smith/original_Golden-Delicious_001.jpg'
    original_path = base_path.rsplit('/', 1)[0] + f'/original_' + base_path.rsplit('/', 1)[1]
    augmented_paths.append(original_path)
    return augmented_paths

# Create a new DataFrame to store the augmented image paths
new_data = []

for index, row in df_train.iterrows():
    class_id = row['Class ID']
    coarse_class_id = row['Coarse Class ID']
    original_image_path = row['Image Path']

    augmented_paths = generate_augmented_paths(original_image_path)

    for path in augmented_paths:
        new_data.append({
            'ClassId': class_id,
            'coarse Class ID': coarse_class_id,
            'image path': path
        })

# Create the new DataFrame
augmented_df = pd.DataFrame(new_data, columns=['ClassId', 'coarse Class ID', 'image path'])

# size of the original dataframe
print(f"Size of the original DataFrame: {df_train.shape[0]}")

# get the size of the augmented dataframe, it should be 8 times the size of the original dataframe
print(f"Size of the augmented DataFrame: {augmented_df.shape[0]}")



Size of the original DataFrame: 2640
Size of the augmented DataFrame: 21120


### **1.3. Preparación de los datos para el entrenamiento**

Hay 2 tareas que deberíamos realizar para preparar nuestros datos:

1. "Aplanar" la imagen de tal manera que simplifiquemos las entradas para el modelo.
2. Normalizar los valores de las imágenes.


In [31]:
import numpy as np

def flatten_and_normalize(images):
    # Aplanar las imágenes
    flattened_images = images.reshape(images.shape[0], -1)

    # Normalizar los valores entre 0 y 1
    normalized_images = flattened_images / 255.0

    return normalized_images

# **2. Modelado**

In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage.io import imread
from skimage.transform import resize

import os
from skimage.io import imread
from skimage.transform import resize

import os
from skimage.io import imread

# Replace train128x128 for train_agumented in the image path
augmented_df['image path'] = augmented_df['image path'].str.replace('train_128x128/', 'train_augmented/')

# Define the function flatten_and_normalize
def flatten_and_normalize(image_url, target_size=(128, 128)):
    try:
        image = imread(image_url)
        image_resized = resize(image, target_size, anti_aliasing=True)
        flattened_image = image_resized.flatten()
        normalized_image = flattened_image / 255.0
        normalized_image = normalized_image.astype(np.float16)
        return normalized_image
    except FileNotFoundError:
        # Modify the path by repeating the subfolder name
        base_path = 'GroceryStoreDataset/dataset/train_augmented/'
        if base_path in image_url:
            path_parts = image_url.split(base_path)
            subfolder, remaining_path = path_parts[1].split('/', 1)
            new_image_url = os.path.join(path_parts[0], base_path, subfolder, subfolder, remaining_path)

            try:
                # Retry with the modified path
                image = imread(new_image_url)
                image_resized = resize(image, target_size, anti_aliasing=True)
                flattened_image = image_resized.flatten()
                normalized_image = flattened_image / 255.0
                normalized_image = normalized_image.astype(np.float16)
                return normalized_image
            except FileNotFoundError:
                # Raise an error if the file is not found even after modification
                if subfolder == 'Brown-Cap-Mushroom':
                  new_image_url = os.path.join(path_parts[0], base_path, 'Mushroom', subfolder, remaining_path)
                  # Retry with the modified path
                  image = imread(new_image_url)
                  image_resized = resize(image, target_size, anti_aliasing=True)
                  flattened_image = image_resized.flatten()
                  normalized_image = flattened_image / 255.0
                  normalized_image = normalized_image.astype(np.float16)
                  return normalized_image
        else:
            raise FileNotFoundError(f"File path does not contain expected base path: {image_url}")


# Aplicar la función a las imágenes en los DataFrames
def preprocess_images(df, image_column):
    # AN ARRAY flatten_and_normalize(url) for url in df[image_column]
    return np.array([flatten_and_normalize(url) for url in df[image_column]])



### **2.1. Modelado a nivel de producto (manzana, aguacate, etc.)**

In [33]:
# Train a Random Forest model with size limitations
rf_classifierProducto = RandomForestClassifier(n_estimators=100,  # Reduce number of trees
                                       max_depth=10,      # Limit tree depth
                                       min_samples_split=5,  # Increase minimum samples for split
                                       n_jobs=-1)           # Use all available cores

# The best results from gridsearch have already been found.
# param_grid = {
#     'n_estimators': [50, 100, 200],         # Number of trees
#     'max_depth': [None, 10, 20, 30],        # Maximum depth of each tree
#     'min_samples_split': [2, 5, 10],        # Minimum number of samples to split an internal node
#     'min_samples_leaf': [1, 2, 4],          # Minimum number of samples required to be at a leaf node
#     'bootstrap': [True, False]              # Whether bootstrap samples are used when building trees
# }

# # Set up the GridSearchCV
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
#                            cv=5, n_jobs=-1, verbose=2)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

In [34]:
X_train = preprocess_images(augmented_df, 'image path')
y_train = augmented_df['coarse Class ID'].values

print("Number of features:", X_train.shape[1], ".That's the number of pixels in the images x3")
print(f"X_train shape: {X_train.shape}")

# Train the model
rf_classifierProducto.fit(X_train, y_train)

print("Training done")

del X_train
del y_train

Number of features: 49152 .That's the number of pixels in the images x3
X_train shape: (21120, 49152)
Training done


In [35]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

X_val = preprocess_images(df_val, 'Image Path')
y_val = df_val['Coarse Class ID'].values

print(f"X_val shape: {X_val.shape}")

# Predictions on validation and test sets
y_val_pred = rf_classifierProducto.predict(X_val)

del X_val

X_val shape: (296, 49152)


In [36]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate Precision, Recall, and F1-Score
precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average='weighted')

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

del y_val_pred
del y_val


Precision: 0.1867
Recall: 0.2500
F1-Score: 0.1671


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **2.2. Modelado a nivel de marca (manzana Golden-Delicious, etc)**

In [37]:
# Train a Random Forest model with size limitations
rf_classifierMarca = RandomForestClassifier(n_estimators=100,  # Reduce number of trees
                                       max_depth=10,      # Limit tree depth
                                       min_samples_split=5,  # Increase minimum samples for split
                                       n_jobs=-1)           # Use all available cores

In [38]:
X_train = preprocess_images(augmented_df, 'image path')
y_train = augmented_df['ClassId'].values

print("Number of features:", X_train.shape[1], ".That's the number of pixels in the images x3")
print(f"X_train shape: {X_train.shape}")

del augmented_df

# Train the model
rf_classifierMarca.fit(X_train, y_train)

print("Training done")

del X_train
del y_train

Number of features: 49152 .That's the number of pixels in the images x3
X_train shape: (21120, 49152)
Training done


In [39]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

X_val = preprocess_images(df_val, 'Image Path')
y_val = df_val['Class ID'].values
del df_val

print(f"X_val shape: {X_val.shape}")

# Predictions on validation and test sets
y_val_pred = rf_classifierMarca.predict(X_val)

del X_val

X_val shape: (296, 49152)


In [40]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate Precision, Recall, and F1-Score
precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average='weighted')

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Precision: 0.1180
Recall: 0.1284
F1-Score: 0.0964


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **3. Análisis de resultados del modelo**

Usaremos los resultados del set de validación para analizar resultados del modelo, y posteriormente el set de prueba para compararlo con nuestro segundo modelo construido.

En el contexto de clasificar productos de un supermercado, la métrica de accuracy (precisión) resulta más relevante que el recall (exactitud) porque nos interesa medir cuántas clasificaciones totales fueron correctas en comparación con todas las predicciones realizadas. El accuracy nos indica el porcentaje de productos que fueron correctamente clasificados del total, lo cual es clave para evaluar el rendimiento general de nuestro modelo en este caso.

Por ejemplo, si clasificamos un conjunto de manzanas y obtenemos un accuracy del 95%, significa que el modelo asignó correctamente el producto a su categoría el 95% de las veces. En cambio, el recall mide cuántos de los productos de una categoría específica fueron identificados correctamente como tal.Esto puede ser menos informativo respecto al rendimiento del modelo en este escenario, ya que un alto recall para una categoría (como decir que de los 10 productos clasificados como aguacates, todos eran aguacates) no garantiza que todos los aguacates presentes en la muestra hayan sido identificados como tales.

Por ejemplo, si hay 100 aguacates en total y el modelo solo clasificó correctamente 10 pero ignoró los otros 90, el recall seria alto (si no se clasificó nada mas como aguacate), lo cual no reflejaría bien el rendimiento general. En cambio, un buen accuracy asegura que el modelo sea consistente en su capacidad de identificar correctamente todos los tipos de productos.

### **3.1. Análisis de resultados de validación**

Los resultados obtenidos del modelo de clasificación no son alentadores. Posteriormente a haber definido el accuracy como métrica principal de evaluación y de realizar un exhaustivo proceso de optimización mediante grid search para los hiperparámetros, así como aplicar técnicas de aumento de datos, el modelo alcanzó un accuracy de apenas 0.18 a nivel de producto (por ejemplo, clasificar entre manzanas, aguacates, etc.) y 0.12 a nivel de subproducto o marca específica (como manzana Golden Delicious o Royal Gala). Estos resultados indican que el modelo tiene un desempeño significativamente limitado en la tarea de clasificación planteada.

La baja capacidad del modelo para capturar tendencias en las imágenes, especialmente a nivel de subproducto, podría deberse a varias razones. En primer lugar, es posible que las características visuales entre diferentes subproductos sean demasiado sutiles o similares, dificultando que el modelo aprenda patrones distintivos. Además, es posible que la representación utilizada para los datos (un arreglo aplandado de los pixeles 128x128 de la imagen, con los canales RGB uno despues del otro) dificute al modelo la tarea de identificar tendencias significativas dentro de las imagenes.

### **3.2. Análisis de resultados de prueba**

Ahora se encontrarán los resultados sobre el conjunto de prueba, notese que estos no se compararán con el otro modelo dentro de este notebook, pero dentro del archivo de analizis de resultados y generación de valor.

In [41]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

X_test = preprocess_images(df_test, 'Image Path')
y_test = df_test['Class ID'].values
del df_test

print(f"X_test shape: {X_test.shape}")

# Predictions on validation and test sets
y_test_pred = rf_classifierMarca.predict(X_test)

del X_test

X_test shape: (2485, 49152)


In [42]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate Precision, Recall, and F1-Score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='weighted')

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Precision: 0.1541
Recall: 0.1815
F1-Score: 0.1353


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Se usará esta metrica de precisión del 0.15 para comaprar nuestro modelo con nuestro segundo modelo construido en nuestro informe de comparación de resultados y generación de valor.

### **3.3. Elementos positivos y oportunidades de mejora**

La calidad del modelo se ve reflejada en sus métricas de accuracy (0.18 a nivel de producto y 0.08 a nivel de subproducto). A pesar de que estos resultados no son excelentes, se pueden identificar algunos aspectos positivos en el proceso de modelado que pudieron haber tenido un impacto en las métricas obtenidas. Entre ellos, la implementación de grid search para optimizar los hiperparámetros y las técnicas de aumento de datos fueron esfuerzos clave para mejorar la generalización del modelo y ampliar su capacidad de clasificación. El uso de grid search permitió identificar configuraciones de hiperparámetros más adecuadas, lo que, en teoría, debería haber ayudado a mejorar la capacidad del modelo para aprender patrones en los datos. La aumentación de datos, por otro lado, contribuyó a diversificar las muestras de entrenamiento y a reducir el riesgo de sobreajuste, lo que es particularmente útil en contextos con imágenes.

Sin embargo, los resultados obtenidos indican que existen varias oportunidades de mejora que pueden abordarse para mejorar la precisión del modelo. Primero, una oportunidad clave sería mejorar la calidad y diversidad de las imágenes utilizadas en el entrenamiento. Incluir más variabilidad en términos de iluminación, perspectivas, resolución y otros factores podría ayudar a que el modelo generalice mejor las características distintivas entre las clases. Además, se podría explorar la posibilidad de analizar si nuestro modelo funciona mejor o peor para algunas clases en particular, y si es así, construir un segundo modelo especializado en clasificar las clases que este primer modelo no puede clasificar correctamente. Esto nos permitiría usar composición de modelos para mejorar la precisión general de nuestro sistema de clasificación.